## BINN Evaluation of Learned Parameters
#### Uses BINNCovasim Neural Network Model

In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

from Modules.Utils.Imports import *
from Modules.Models.BuildBINNs import BINNCovasim
from Modules.Utils.ModelWrapper import ModelWrapper

import Modules.Utils.PDESolver as PDESolver
import Modules.Loaders.DataFormatter as DF
from utils import get_case_name #, AIC_OLS, RSS

# helper functions
def to_torch(x):
    return torch.from_numpy(x).float().to(device)
def to_numpy(x):
    return x.detach().cpu().numpy()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Covasim 3.1.4 (2022-10-22) — © 2020-2022 by IDM


In [2]:
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

# instantiate BINN model parameters and path
path = '../Data/covasim_data/drums_data/'
# path = '../Data/covasim_data/xin_data/'

population = 50000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic = True
masking = False
multiple = True
n_runs = 1000
chi_type = 'piecewise'

case_name = get_case_name(population, test_prob, trace_prob, keep_d, dynamic=dynamic, chi_type=chi_type)
# yita_lb, yita_ub = 0.2, 0.4

Device set to cpu


In [3]:
if masking:
    case_name = case_name + '_masking'
if multiple:
    params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name + '_' + str(n_runs), plot=False)
else:
    params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name, plot=False)

In [4]:
# split into train/val and convert to torch
if multiple:
    data = np.mean(params['data'], axis=0)
    data = (data / params['population'])
else:
    data = params['data']
    data = (data / params['population']).to_numpy()

N = len(data)
t_max = N - 1
t = np.arange(N)[:,None]

params.pop('data')

tracing_array = params['tracing_array']

In [5]:
# mydir = '../models/covasim/2023-06-21_00-12-29' # piecewise h(t) function
# mydir = '../models/covasim/2023-06-25_23-26-00' # constant h(t) function
# mydir = '../models/covasim/2023-06-26_23-48-27' # constant h(t) function
mydir = '../models/covasim/2023-07-06_23-47-16' # piecewise h(t) function, no masking

In [6]:
# instantiate BINN model
binn = BINNCovasim(params, t_max, tracing_array, keep_d=keep_d).to(device)

In [7]:
parameters = binn.parameters()
model = ModelWrapper(binn, None, None, save_name=os.path.join(mydir, case_name))

In [8]:
# load model weights. if retrain==True then load the retrained model
if retrain:
    model.save_name += '_retrain'
model.save_name += '_best_val'
model.load(model.save_name + '_model', device=device)

# grab initial condition
u0 = data[0, :].copy()

In [9]:
# learned surface fitter
def surface_fitter(t):
    res = binn.surface_fitter(t)
    return res

# learned contact_rate function
def contact_rate(u):
    res = binn.eta_func(to_torch(u)) # [:,[0,3,4]]
    return to_numpy(res)

# learned effective tracing rate function
def beta(u):
    res = binn.beta_func(to_torch(u))
    return to_numpy(res)

# learned diagnosis of quarantined rate function
def tau(u):
    res = binn.tau_func(to_torch(u))
    return to_numpy(res)

# do regression to figure out contact rate
def contact_rate_regression(u):
    s, a, y = u[:, 0][:, None], u[:, 1][:, None], u[:, 2][:, None]
    features = [np.ones_like(a), s, s**2, a, y] #
    features = np.concatenate(features, axis=1)
    res = features @ regression_coefs_cr
    # res *= 1.4
    return res

# do regression to figure out tracing rate
def beta_regression(u):
    a, b = u[:, 0][:, None], u[:, 1][:, None]
    features = [np.ones_like(a), a, b] #
    features = np.concatenate(features, axis=1)
    res = features @ regression_coefs_qt
    return res

# do regression to figure out diagnoses rate (on quarantined folks)
def tau_regression(u):
    a, b = u[:, 0][:, None], u[:, 1][:, None]
    features = [np.ones_like(a), a, b] #
    features = np.concatenate(features, axis=1)
    res = features @ regression_coefs_tau
    return res


In [10]:
t_torch = to_torch(t)
solutions = surface_fitter(t_torch).detach().numpy()
eta_values = contact_rate(data[:,[0, 3, 4]])
# beta_values = beta(np.sum(data[:,[0, 3, 4]], axis=1))
tau_values = tau(data[:,[3, 4]])
max_T = np.max(data[:,1])

max_S_nn = np.max(solutions[:,0])
max_T_nn = np.max(solutions[:,1])
max_E_nn = np.max(solutions[:,2])
max_A_nn = np.max(solutions[:,3])
max_Y_nn = np.max(solutions[:,4])
max_Q_nn = np.max(solutions[:,5])
max_D_nn = np.max(solutions[:,6])
max_R_nn = np.max(solutions[:,7])
max_F_nn = np.max(solutions[:,8])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [11]:
# simulate PDE. First grab the parameter values from the loaded BINN model.
params['yita_lb'] = model.model.yita_lb
params['yita_ub'] = model.model.yita_ub
params['beta_lb'] = model.model.beta_lb
params['beta_ub'] = model.model.beta_ub
params['tau_lb'] = model.model.tau_lb
params['tau_ub'] = model.model.tau_ub

In [12]:
# if D is to be included in the model, otherwise use the other solver.
if keep_d:
    RHS = PDESolver.STEAYDQRF_RHS_dynamic
    u_sim_NN = PDESolver.STEAYDQRF_sim(RHS, u0, t, contact_rate, beta, tau, params, chi_type, analyze_beta = False, analyze_eta = False, analyze_tau = False)
    u_sim_NN *= population
    
    # commented out evaluations with regression models
    #u_sim_regression = PDESolver.STEAYDQRF_sim(RHS, u0, t, contact_rate_regression, beta_regression, tau_regression, params, chi_type)
    #u_sim_regression *= population

Traceback (most recent call last):
capi_return is NULL
Call-back cb_fcn_in___user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
  File "/Users/jordanklein22/Documents/GitHub/COVASIM_EQL_BINNS/Notebooks/../Modules/Utils/PDESolver.py", line 356, in RHS_ty
    return RHS(t, y, contact_rate, quarantine_test, tau, params, t_max, chi_type, analyze_beta, analyze_eta, analyze_tau)
  File "/Users/jordanklein22/Documents/GitHub/COVASIM_EQL_BINNS/Notebooks/../Modules/Utils/PDESolver.py", line 318, in STEAYDQRF_RHS_dynamic
    return np.array([ds, dt, de, da, dy, dd, dq, dr, df])
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9,) + inhomogeneous part.


: 

: 

In [ ]:
# rescale data values to represent number of people in state X rather than ratios
data *= population

plot=True
if plot:
    # data = params['data']
    n = data.shape[1]
    col_names = list('STEAYDQRF') if keep_d else list('STEAYQRF')
    # t = np.arange(1, data.shape[0] + 1)
    # plot compartments
    fig = plt.figure(figsize=(15, 15))
    for i in range(1, n + 1):
        ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
        ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
        ax.plot(t, u_sim_NN[:, i - 1], '-*r', label='ODE-NN')
        ax.set_title(col_names[i - 1])
        ax.legend(fontsize=8)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        plt.tight_layout(pad=2)
        plt.savefig(os.path.join(model.save_folder, case_name + '.png') )
        # plt.show()

In [ ]:
# %% optional, add one comparison based on the results ODE-DE
de_res_file = 'fitted_' + case_name + '.csv'
ode_de_df = pd.read_csv('../Data/covasim_data/xin_data/' + de_res_file, index_col=0)
ode_de_array = ode_de_df.to_numpy()
# plot compartments
plt.rcParams.update({'font.size': 12})
fig = plt.figure(figsize=(18, 9))
for i in range(1, n):
    ax = fig.add_subplot(2, 4, i) # int(np.ceil(n / 4))
    ax.plot(t, data[:, i - 1], '.k', label='Test-ABM')
    ax.plot(t, u_sim_NN[:, i - 1], 'r.-', label='ODE-NN')
    ax.plot(t, ode_de_array[:, i - 1], 'g.-', label='ODE-DE')
    if i > 5:
        ax.set_xlabel("Time (Days)")
    if i % 3 == 1:
        ax.set_ylabel("Count")
    ax.set_title(col_names[i - 1])
    if i == 1:
        ax.legend(loc="best")
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.tight_layout(pad=2)
    plt.savefig(os.path.join(model.save_folder, case_name + '_without_F' + '.png'), dpi=300)
    # plt.show()

In [ ]:
# %% save without R
fig = plt.figure(figsize=(18, 9))
for i in range(1, n + 1):
    if i == n - 1:
        continue
    ax = fig.add_subplot(2, 4, i) if i < n else fig.add_subplot(2, 4, i - 1) # int(np.ceil(n / 4))
    ax.plot(t, data[:, i - 1], '.k', label='Test-ABM')
    ax.plot(t, u_sim_NN[:, i - 1], 'r.-', label='ODE-NN')
    ax.plot(t, ode_de_array[:, i - 1], 'g.-', label='ODE-DE')
    if i > 4:
        ax.set_xlabel("Time (Days)")
    if i % 4 == 1:
        ax.set_ylabel("Count")
    ax.set_title(col_names[i - 1])
    if i == 1:
        ax.legend(loc="best")
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.tight_layout(pad=2)
    plt.savefig(os.path.join(model.save_folder, case_name + '_without_R' + '.png'), dpi=300)

In [ ]:
#%% RMSE
from sklearn.metrics import mean_squared_error
rmse_NN = {}
rmse_SR = {}
rmse_DE = {}
for i, col in enumerate(col_names):
    res = mean_squared_error(data[:, i], u_sim_NN[:,i], squared=False)
    rmse_NN[col] = res
    rmse_SR[col] = res
    res = mean_squared_error(data[:, i], ode_de_array[:, i], squared=False)
    rmse_DE[col] = res
rmse_df = pd.DataFrame([rmse_NN, rmse_SR, rmse_DE], index=['ODE-NN', 'ODE-SR', 'ODE-DE'])
rmse_df['Mean'] = rmse_df.mean(axis=1)
rmse_df.to_csv(os.path.join(model.save_folder, case_name + '_rmse' + '.csv'))

In [ ]:
#%% NRMSE
from sklearn.metrics import mean_squared_error
nrmse_NN = {}
nrmse_SR = {}
nrmse_DE = {}
for i, col in enumerate(col_names):
    y_max, y_min = max(data[:, i]), min(data[:, i])
    res = mean_squared_error(data[:, i], u_sim_NN[:,i], squared=False) / (y_max - y_min)
    nrmse_NN[col] = res
    nrmse_SR[col] = res
    res = mean_squared_error(data[:, i], ode_de_array[:, i], squared=False) / (y_max - y_min)
    nrmse_DE[col] = res
nrmse_df = pd.DataFrame([nrmse_NN, nrmse_SR, nrmse_DE], index=['ODE-NN', 'ODE-SR', 'ODE-DE'])
nrmse_df['Mean'] = nrmse_df.mean(axis=1)
nrmse_df.to_csv(os.path.join(model.save_folder, case_name + '_nrmse' + '.csv'))